In [1]:
class CrossingPuzzle:
    def __init__(self):
        # Times of people on the left bank initially
        self.left = {'Amogh': 5, 'Ameya': 10, 'Grandmother': 20, 'Grandfather': 25}
        self.right = {}
        self.umbrella = 'left'  # Umbrella starts on left side
        self.time = 0
        self.time_limit = 60

    def goal_test(self):
        # Goal: everyone is on the right side within time limit
        return len(self.left) == 0 and self.time <= self.time_limit

    def move_gen(self):
        # Generate all valid moves from current state
        moves = []
        if self.umbrella == 'left':
            # Move 1 or 2 people from left to right
            people = list(self.left.items())
            n = len(people)
            for i in range(n):
                # One person crosses
                name1, t1 = people[i]
                time_taken = t1
                if self.time + time_taken > self.time_limit:
                    continue
                moves.append(([name1], time_taken))

                for j in range(i+1, n):
                    # Two people cross
                    name2, t2 = people[j]
                    time_taken = max(t1, t2)
                    if self.time + time_taken > self.time_limit:
                        continue
                    moves.append(([name1, name2], time_taken))
        else:
            # Move one person from right to left (umbrella returns)
            for name, t in self.right.items():
                time_taken = t
                if self.time + time_taken > self.time_limit:
                    continue
                moves.append(([name], time_taken))
        return moves

    def do_move(self, move):
        # move = ([names], time_taken)
        names, time_taken = move
        if self.umbrella == 'left':
            # Move people from left to right
            for name in names:
                self.right[name] = self.left.pop(name)
            self.umbrella = 'right'
        else:
            # Move people from right to left
            for name in names:
                self.left[name] = self.right.pop(name)
            self.umbrella = 'left'
        self.time += time_taken

    def undo_move(self, move):
        # Reverse the move, used for backtracking
        names, time_taken = move
        if self.umbrella == 'right':
            # Undo move from left to right
            for name in names:
                self.left[name] = self.right.pop(name)
            self.umbrella = 'left'
        else:
            # Undo move from right to left
            for name in names:
                self.right[name] = self.left.pop(name)
            self.umbrella = 'right'
        self.time -= time_taken


def solve():
    puzzle = CrossingPuzzle()
    path = []

    def backtrack():
        if puzzle.goal_test():
            return True
        for move in puzzle.move_gen():
            puzzle.do_move(move)
            path.append(move)
            if backtrack():
                return True
            path.pop()
            puzzle.undo_move(move)
        return False

    if backtrack():
        print("Solution found in", len(path), "moves:")
        side = 'left'
        time = 0
        for i, (names, t) in enumerate(path, 1):
            dir_arrow = "→" if side == 'left' else "←"
            print(f"Step {i}: {', '.join(names)} cross {dir_arrow} ({t} min)")
            time += t
            side = 'right' if side == 'left' else 'left'
        print(f"Total time: {time} minutes")
    else:
        print("No solution found within time limit.")

solve()


Solution found in 5 moves:
Step 1: Amogh, Ameya cross → (10 min)
Step 2: Amogh cross ← (5 min)
Step 3: Grandfather, Grandmother cross → (25 min)
Step 4: Ameya cross ← (10 min)
Step 5: Amogh, Ameya cross → (10 min)
Total time: 60 minutes
